<a href="https://colab.research.google.com/github/hoohahaBIGHEAD/AI/blob/main/%EB%94%B0%EB%A6%89%EC%9D%B4(%EC%A0%95%EB%A6%AC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

선형회귀 모델
1. Linear Regression
2. Ridge
3. Lasso
4. ElasticNet
5. Decision Tree
6. Randomforest
7. XGBoostRegression
8. LightGBMRegression
9. ExtratreeRegression
10. AdaboostRegression

#데이터 준비

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!cp /gdrive/MyDrive/temp/data.zip ./

In [ ]:
!unzip data.zip

Archive:  data.zip
  inflating: data/submission.csv     
  inflating: data/test.csv           
  inflating: data/train.csv          


In [ ]:
train = pd.read_csv('data/train.csv') 
test = pd.read_csv('data/test.csv')
train_pro = pd.read_csv('data/train.csv')
test_pro = pd.read_csv('data/test.csv')
submission = pd.read_csv('/content/data/submission.csv')

#EDA/데이터 전처리


In [ ]:
bike_data = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [ ]:
############
option = True
bike_data['isTrain'] = True
test['isTrain'] = False

total = pd.concat([bike_data, test])

##결측치처리

In [ ]:
############
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

it_train = total.copy()
it_train = IterativeImputer(random_state=1).fit_transform(it_train)

itImp = pd.DataFrame(it_train)
itImp.columns = total.columns

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


In [ ]:
###########
itImp_mid = itImp.copy()
col_name = total.columns.drop(['hour_bef_precipitation'])
for iIt in col_name:
  Q1=itImp_mid[iIt].quantile(0.25)
  Q3=itImp_mid[iIt].quantile(0.75)
  IQR = Q3-Q1
  for i in range(2174):
    if itImp_mid[iIt][i] < (Q1 - 1.5*IQR):
      itImp_mid[iIt][i] = (Q1 - 1.5*IQR)
    elif itImp_mid[iIt][i] > (Q3 + 1.5*IQR):
      itImp_mid[iIt][i] = (Q3 + 1.5*IQR)

##더미변수추가(출퇴근 시간 분류)

In [ ]:
def busyHourGen(data, col):
  lst = data[col]
  lst_ = []
  for i in lst:
    if (6 < i < 10) or (16 < i < 20):
      lst_.append(1)
    else:
      lst_.append(0)
  data['busy_hour'] = lst_
  return data

In [ ]:
################# 데이터 조정 후 분리
an_bef = busyHourGen(itImp_mid, 'hour')

tmp = an_bef[(an_bef['isTrain'] == False)]
an = an_bef.drop(tmp.index, axis=0).copy()
an.drop(['isTrain'], axis=1, inplace=True)

tmp = an_bef[(an_bef['isTrain'] == True)]
test = an_bef.drop(tmp.index, axis=0).copy()
test.drop(['isTrain', 'count'], axis=1, inplace=True)

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
X = an[an.columns.difference(['count', 'hour_bef_visibility'])]
X = np.column_stack((X['hour']**5, X['hour_bef_temperature']**4, X))
y = an['count']

b_train_X, b_val_X, b_train_y, b_val_y = train_test_split(X, y, shuffle = True, random_state=2021, test_size=0.15)

##Polynomial

In [ ]:
##############
#독립변수, 종속변수 설정
X = an[an.columns.difference(['id', 'count', 'hour_bef_visibility'])]
colls = X.columns.tolist()
if option:
  X = np.column_stack((X['hour']**5, X['hour_bef_temperature']**4, X))
y = an['count']

#테스트셋 전처리(결측치, 더미 변수 추가)
it_test = test[test.columns.difference(['id', 'hour_bef_visibility'])].copy()
it_test = busyHourGen(it_test, 'hour')
it_test = it_test[colls]
if option:
  X_test = np.column_stack((it_test['hour']**5, it_test['hour_bef_temperature']**4, it_test))
X_test = IterativeImputer(random_state=1).fit_transform(X_test)

it_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 715 entries, 1459 to 2173
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   busy_hour               715 non-null    int64  
 1   hour                    715 non-null    float64
 2   hour_bef_humidity       715 non-null    float64
 3   hour_bef_ozone          715 non-null    float64
 4   hour_bef_pm10           715 non-null    float64
 5   hour_bef_pm2.5          715 non-null    float64
 6   hour_bef_precipitation  715 non-null    float64
 7   hour_bef_temperature    715 non-null    float64
 8   hour_bef_windspeed      715 non-null    float64
dtypes: float64(8), int64(1)
memory usage: 55.9 KB


In [ ]:
train

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0
...,...,...,...,...,...,...,...,...,...,...,...
1454,2174,4,16.8,0.0,1.6,53.0,2000.0,0.031,37.0,27.0,21.0
1455,2175,3,10.8,0.0,3.8,45.0,2000.0,0.039,34.0,19.0,20.0
1456,2176,5,18.3,0.0,1.9,54.0,2000.0,0.009,30.0,21.0,22.0
1457,2178,21,20.7,0.0,3.7,37.0,1395.0,0.082,71.0,36.0,216.0


In [ ]:
test

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,busy_hour
1459,0.0,7.0,20.7,0.0,1.3,62.0,954.0,0.041000,44.000000,27.000000,1
1460,1.0,17.0,30.0,0.0,5.4,33.0,1590.0,0.061000,49.000000,36.000000,1
1461,2.0,13.0,19.0,1.0,2.1,95.0,193.0,0.020000,36.000000,28.000000,0
1462,4.0,6.0,22.5,0.0,2.5,60.0,1185.0,0.027000,52.000000,38.000000,0
1463,5.0,22.0,14.6,1.0,3.4,93.0,218.0,0.041000,18.000000,15.000000,0
...,...,...,...,...,...,...,...,...,...,...,...
2169,2148.0,1.0,24.6,0.0,2.4,60.0,1745.0,0.037603,19.270062,14.890008,0
2170,2149.0,1.0,18.1,0.0,1.0,55.0,2000.0,0.022336,20.392692,13.666635,0
2171,2165.0,9.0,23.3,0.0,2.3,66.0,1789.0,0.020000,17.000000,15.000000,1
2172,2166.0,16.0,27.0,0.0,1.6,46.0,1956.0,0.032000,40.000000,26.000000,0


##base 결측치처리

In [ ]:
train.fillna(0,inplace = True)
test.fillna(0,inplace = True)
train_pro.fillna(0,inplace = True)
test_pro.fillna(0,inplace = True)

In [ ]:
train.isnull().sum()

id                        0
hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
count                     0
dtype: int64

In [ ]:
test_pro.isnull().sum()

id                        0
hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
dtype: int64

##출퇴근시간 더비변수 적용

In [ ]:
train = busyHourGen(train, 'hour')
test = busyHourGen(test, 'hour')
train_pro = busyHourGen(train, 'hour')
test_pro = busyHourGen(test, 'hour')

##상관관계가 작은 hour_bef_visibility 제거 

In [ ]:
train_pro = train_pro.drop(['hour_bef_visibility'],axis = 1)
test_pro = test_pro.drop(['hour_bef_visibility'],axis = 1)
train_pro

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count,busy_hour
0,3,20,16.3,1.0,1.5,89.0,0.027,76.0,33.0,49.0,0
1,6,13,20.1,0.0,1.4,48.0,0.042,73.0,40.0,159.0,0
2,7,6,13.9,0.0,0.7,79.0,0.033,32.0,19.0,26.0,0
3,8,23,8.1,0.0,2.7,54.0,0.040,75.0,64.0,57.0,0
4,9,18,29.5,0.0,4.8,7.0,0.057,27.0,11.0,431.0,1
...,...,...,...,...,...,...,...,...,...,...,...
1454,2174,4,16.8,0.0,1.6,53.0,0.031,37.0,27.0,21.0,0
1455,2175,3,10.8,0.0,3.8,45.0,0.039,34.0,19.0,20.0,0
1456,2176,5,18.3,0.0,1.9,54.0,0.009,30.0,21.0,22.0,0
1457,2178,21,20.7,0.0,3.7,37.0,0.082,71.0,36.0,216.0,0


In [ ]:
test_pro

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,busy_hour
1459,0.0,7.0,20.7,0.0,1.3,62.0,0.041000,44.000000,27.000000,1
1460,1.0,17.0,30.0,0.0,5.4,33.0,0.061000,49.000000,36.000000,1
1461,2.0,13.0,19.0,1.0,2.1,95.0,0.020000,36.000000,28.000000,0
1462,4.0,6.0,22.5,0.0,2.5,60.0,0.027000,52.000000,38.000000,0
1463,5.0,22.0,14.6,1.0,3.4,93.0,0.041000,18.000000,15.000000,0
...,...,...,...,...,...,...,...,...,...,...
2169,2148.0,1.0,24.6,0.0,2.4,60.0,0.037603,19.270062,14.890008,0
2170,2149.0,1.0,18.1,0.0,1.0,55.0,0.022336,20.392692,13.666635,0
2171,2165.0,9.0,23.3,0.0,2.3,66.0,0.020000,17.000000,15.000000,1
2172,2166.0,16.0,27.0,0.0,1.6,46.0,0.032000,40.000000,26.000000,0


##train, test set 구분

In [ ]:
df_shuffled=sklearn.utils.shuffle(train)
df_shuffled

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count,busy_hour
354,542,14,14.6,0.0,6.7,14.0,2000.0,0.048,59.0,11.0,93.0,0
1019,1530,10,14.4,0.0,2.1,49.0,1581.0,0.025,49.0,25.0,82.0,0
986,1487,21,14.7,0.0,1.6,31.0,1175.0,0.052,77.0,68.0,138.0,0
954,1449,4,15.8,0.0,2.3,84.0,387.0,0.047,68.0,36.0,23.0,0
999,1502,2,13.6,0.0,2.2,70.0,491.0,0.039,69.0,39.0,21.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1025,1541,3,16.3,0.0,2.4,59.0,792.0,0.018,111.0,65.0,25.0,0
881,1339,22,14.7,0.0,2.3,68.0,452.0,0.047,78.0,50.0,118.0,0
144,213,21,10.5,0.0,2.6,17.0,2000.0,0.038,58.0,15.0,122.0,0
866,1316,16,20.3,0.0,4.8,34.0,1328.0,0.057,71.0,29.0,170.0,0


In [ ]:
df_shuffled_pro=sklearn.utils.shuffle(train_pro)
df_shuffled_pro

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count,busy_hour
230,361,23,16.8,0.0,2.2,45.0,0.030,31.0,17.0,123.0,0
1151,1739,2,17.1,0.0,0.8,53.0,0.026,108.0,65.0,28.0,0
982,1483,11,23.5,0.0,3.0,33.0,0.030,50.0,32.0,108.0,0
1449,2169,4,10.2,0.0,1.3,63.0,0.033,58.0,29.0,11.0,0
1198,1809,2,17.1,0.0,1.7,57.0,0.020,46.0,29.0,70.0,0
...,...,...,...,...,...,...,...,...,...,...,...
1341,2020,23,10.9,0.0,1.6,45.0,0.036,80.0,73.0,62.0,0
1081,1627,9,16.6,0.0,2.7,44.0,0.029,89.0,50.0,108.0,1
907,1374,10,11.1,0.0,3.9,18.0,0.027,55.0,12.0,77.0,0
53,87,2,11.4,0.0,2.1,40.0,0.033,103.0,24.0,23.0,0


In [ ]:
p_train = df_shuffled.iloc[150:, :]
p_test = df_shuffled.iloc[:150, :]
pp_train = df_shuffled_pro.iloc[150:, :]
pp_test = df_shuffled_pro.iloc[:150, :]


train 전체 set


In [ ]:
final_x = train_pro.drop(['count'], axis = 1)
final_y = train_pro['count']

train - test set을 나누는 다른 코드


In [ ]:
f_train_X, f_val_X, f_train_y, f_val_y = train_test_split(final_x, final_y, shuffle = True, random_state=2021, test_size=0.33)

In [ ]:
f_train_X

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,busy_hour
1346,2028,17,25.7,0.0,3.5,35.0,0.069,51.0,27.0,1
1273,1928,1,10.6,0.0,0.2,71.0,0.000,0.0,0.0,0
1382,2073,15,19.6,0.0,4.1,53.0,0.041,39.0,13.0,0
720,1100,5,9.2,0.0,1.8,53.0,0.036,31.0,18.0,0
395,601,19,15.9,0.0,3.5,47.0,0.043,79.0,25.0,1
...,...,...,...,...,...,...,...,...,...,...
621,952,11,24.0,0.0,1.7,34.0,0.019,72.0,42.0,0
1152,1740,14,27.9,0.0,3.8,27.0,0.065,51.0,30.0,0
57,91,10,12.1,0.0,1.5,47.0,0.009,90.0,25.0,0
1365,2052,15,27.4,0.0,3.8,23.0,0.067,47.0,22.0,0


In [ ]:
final_x

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,busy_hour
0,3,20,16.3,1.0,1.5,89.0,0.027,76.0,33.0,0
1,6,13,20.1,0.0,1.4,48.0,0.042,73.0,40.0,0
2,7,6,13.9,0.0,0.7,79.0,0.033,32.0,19.0,0
3,8,23,8.1,0.0,2.7,54.0,0.040,75.0,64.0,0
4,9,18,29.5,0.0,4.8,7.0,0.057,27.0,11.0,1
...,...,...,...,...,...,...,...,...,...,...
1454,2174,4,16.8,0.0,1.6,53.0,0.031,37.0,27.0,0
1455,2175,3,10.8,0.0,3.8,45.0,0.039,34.0,19.0,0
1456,2176,5,18.3,0.0,1.9,54.0,0.009,30.0,21.0,0
1457,2178,21,20.7,0.0,3.7,37.0,0.082,71.0,36.0,0


In [ ]:
final_y

0        49.0
1       159.0
2        26.0
3        57.0
4       431.0
        ...  
1454     21.0
1455     20.0
1456     22.0
1457    216.0
1458    170.0
Name: count, Length: 1459, dtype: float64

In [ ]:
p_train

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count,busy_hour
784,1200,8,3.2,0.0,0.4,83.0,728.0,0.004,30.0,0.0,21.0,1
268,415,17,24.1,0.0,2.8,53.0,525.0,0.082,88.0,51.0,158.0,1
524,811,7,13.9,0.0,0.7,54.0,2000.0,0.006,42.0,26.0,106.0,1
45,77,17,25.1,0.0,4.5,33.0,2000.0,0.060,32.0,21.0,268.0,1
420,649,16,20.5,0.0,4.2,36.0,2000.0,0.055,20.0,14.0,302.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1025,1541,3,16.3,0.0,2.4,59.0,792.0,0.018,111.0,65.0,25.0,0
881,1339,22,14.7,0.0,2.3,68.0,452.0,0.047,78.0,50.0,118.0,0
144,213,21,10.5,0.0,2.6,17.0,2000.0,0.038,58.0,15.0,122.0,0
866,1316,16,20.3,0.0,4.8,34.0,1328.0,0.057,71.0,29.0,170.0,0


In [ ]:
pp_train

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count,busy_hour
749,1146,22,12.2,0.0,2.5,76.0,0.045,58.0,29.0,93.0,0
883,1342,20,17.3,0.0,3.8,47.0,0.052,47.0,16.0,155.0,0
496,763,5,16.6,0.0,1.6,73.0,0.017,39.0,27.0,20.0,0
792,1209,22,8.6,0.0,1.7,52.0,0.049,72.0,64.0,60.0,0
137,203,13,19.3,0.0,3.2,52.0,0.011,126.0,37.0,148.0,0
...,...,...,...,...,...,...,...,...,...,...,...
1341,2020,23,10.9,0.0,1.6,45.0,0.036,80.0,73.0,62.0,0
1081,1627,9,16.6,0.0,2.7,44.0,0.029,89.0,50.0,108.0,1
907,1374,10,11.1,0.0,3.9,18.0,0.027,55.0,12.0,77.0,0
53,87,2,11.4,0.0,2.1,40.0,0.033,103.0,24.0,23.0,0


data : train_x, y
processed data : train_pro_x, y

In [ ]:
train_x = p_train.drop(['count'],axis = 1)
train_y = p_train['count']
test_x = p_test.drop(['count'],axis = 1)
test_y = p_test['count']
train_pro_x = pp_train.drop(['count'],axis = 1)
train_pro_y = pp_train['count']
test_pro_x = pp_test.drop(['count'],axis = 1)
test_pro_y = pp_test['count']

In [ ]:
train_pro_x

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,busy_hour
749,1146,22,12.2,0.0,2.5,76.0,0.045,58.0,29.0,0
883,1342,20,17.3,0.0,3.8,47.0,0.052,47.0,16.0,0
496,763,5,16.6,0.0,1.6,73.0,0.017,39.0,27.0,0
792,1209,22,8.6,0.0,1.7,52.0,0.049,72.0,64.0,0
137,203,13,19.3,0.0,3.2,52.0,0.011,126.0,37.0,0
...,...,...,...,...,...,...,...,...,...,...
1341,2020,23,10.9,0.0,1.6,45.0,0.036,80.0,73.0,0
1081,1627,9,16.6,0.0,2.7,44.0,0.029,89.0,50.0,1
907,1374,10,11.1,0.0,3.9,18.0,0.027,55.0,12.0,0
53,87,2,11.4,0.0,2.1,40.0,0.033,103.0,24.0,0


In [ ]:
test_pro_y

230     123.0
1151     28.0
982     108.0
1449     11.0
1198     70.0
        ...  
901      98.0
1242     87.0
1101     17.0
344       8.0
223     105.0
Name: count, Length: 150, dtype: float64

#모델링

## Template

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model as lm
import xgboost
from scipy.stats import uniform, randint

##RandomForest
# data 차이파악

In [ ]:
model=RandomForestRegressor(n_estimators=20)
model.fit(train_x, train_y)
pred = model.predict(test_x)
RMSE = mean_squared_error(test_y, pred)**0.5
print(RMSE)

35.59927878482934


In [ ]:
model=RandomForestRegressor(n_estimators=20)
model.fit(train_pro_x, train_pro_y)
pred = model.predict(test_pro_x)
RMSE = mean_squared_error(test_pro_y, pred)**0.5
print(RMSE)

34.04073882864471


##XGB
data 차이파악

In [ ]:
xgb_model = xgboost.XGBRegressor(n_estimators = 100)
xgb_model.fit(train_x,train_y)
pred = xgb_model.predict(test_x)
RMSE = mean_squared_error(test_y, pred)**0.5
print(RMSE)

[03:02:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
35.999625256039025


In [ ]:
xgb_model = xgboost.XGBRegressor(n_estimators = 100)
xgb_model.fit(train_pro_x,train_pro_y)
pred = xgb_model.predict(test_pro_x)
RMSE = mean_squared_error(test_pro_y, pred)**0.5
print(RMSE)

[03:02:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
35.96162569538966


#하이퍼 파라미터

하이퍼 파라미터 방법

1. Manual Search
2. Grid Search
3. Random Search
4. Bayesian Optimization
5. Non-Probabilistic
6. Evolutionary Optimization
7. Gradient-based Optimization
8. Early Stopping

하이퍼 파라미터 framework

**optuna**



extra randomforest

In [ ]:
pip install optuna

     |████████████████████████████████| 308 kB 9.5 MB/s 
     |████████████████████████████████| 209 kB 56.2 MB/s 
     |████████████████████████████████| 80 kB 8.8 MB/s 
     |████████████████████████████████| 75 kB 4.7 MB/s 
     |████████████████████████████████| 112 kB 56.4 MB/s 
     |████████████████████████████████| 49 kB 6.1 MB/s 
     |████████████████████████████████| 149 kB 54.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=d81173792f1fcb0789e1f671cc0f90181a87d1bcb5d723c4ddbd49d9d1e63eb4
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import SGDClassifier
import optuna

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
import optuna
import sklearn

##ExtraTreesRegressor

ET 모델

In [ ]:
  # model = ExtraTreesRegressor(**paramset)
  # model.fit(f_train_X, f_train_y)
  # pred = model.predict(f_val_X)
  # rmse = np.sqrt(mean_squared_error(f_val_y, pred))

In [ ]:
def extratree(trial):
  paramset = {
      "n_estimators" : trial.suggest_int("n_estimators", 100, 1000, step = 1),
      "max_depth" : trial.suggest_int("max_depth", 2, 20),
      "min_samples_split":trial.suggest_int("min_samples_split", 2, 4),
      "min_samples_leaf":trial.suggest_int("min_samples_leaf", 1, 3),
      "max_features":trial.suggest_categorical("max_features", ["auto", "sqrt", "log2", 4,5,6]),
      "warm_start":trial.suggest_categorical("warm_start", [True, False]),
      "random_state":2021
  }
  model = ExtraTreesRegressor(**paramset)
  model.fit(train_x, train_y)
  pred = model.predict(test_x)
  rmse = np.sqrt(mean_squared_error(test_y, pred))
  return rmse

In [ ]:
def extratree_p(trial):
  paramset = {
      "n_estimators" : trial.suggest_int("n_estimators", 100, 1000, step = 1),
      "max_depth" : trial.suggest_int("max_depth", 2, 20),
      "min_samples_split":trial.suggest_int("min_samples_split", 2, 4),
      "min_samples_leaf":trial.suggest_int("min_samples_leaf", 1, 3),
      "max_features":trial.suggest_categorical("max_features", ["auto", "sqrt", "log2", 4,5,6]),
      "warm_start":trial.suggest_categorical("warm_start", [True, False]),
      "random_state":2021
  }
  model = ExtraTreesRegressor(**paramset)
  model.fit(train_pro_x, train_pro_y)
  pred = model.predict(test_pro_x)
  rmse = np.sqrt(mean_squared_error(test_pro_y, pred))
  return rmse

In [ ]:
def extratree_p(trial):
  paramset = {
      "n_estimators" : trial.suggest_int("n_estimators", 100, 1000, step = 1),
      "max_depth" : trial.suggest_int("max_depth", 2, 20),
      "min_samples_split":trial.suggest_int("min_samples_split", 2, 4),
      "min_samples_leaf":trial.suggest_int("min_samples_leaf", 1, 3),
      "max_features":trial.suggest_categorical("max_features", ["auto", "sqrt", "log2", 4,5,6]),
      "warm_start":trial.suggest_categorical("warm_start", [True, False]),
      "random_state":2021
  }
  model = ExtraTreesRegressor(**paramset)
  model.fit(train_pro_x, train_pro_y)
  pred = model.predict(test_pro_x)
  rmse = np.sqrt(mean_squared_error(test_pro_y, pred))
  return rmse

In [ ]:
sampler = TPESampler(seed = 2021)
study = optuna.create_study(
    study_name = "et_optimizer",
    direction = "minimize",
    sampler = sampler,
)
study.optimize(extratree, n_trials = 30)

[I 2022-01-22 03:03:07,645] A new study created in memory with name: et_optimizer
[I 2022-01-22 03:03:09,990] Trial 0 finished with value: 34.21895599672502 and parameters: {'n_estimators': 645, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'warm_start': True}. Best is trial 0 with value: 34.21895599672502.
[I 2022-01-22 03:03:11,545] Trial 1 finished with value: 34.83976056688927 and parameters: {'n_estimators': 967, 'max_depth': 13, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'warm_start': True}. Best is trial 0 with value: 34.21895599672502.
[I 2022-01-22 03:03:12,596] Trial 2 finished with value: 34.88389127208869 and parameters: {'n_estimators': 625, 'max_depth': 11, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 5, 'warm_start': False}. Best is trial 0 with value: 34.21895599672502.
[I 2022-01-22 03:03:13,274] Trial 3 finished with value: 34.49359028125019 and parameters: {'n_estimators': 371, 'm

하이퍼 파라미터 튜닝 BEST


{'n_estimators': 478, 'max_depth': 19, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'auto', 'warm_start': True}. Best is trial 20 with value: 31.828042991481777.

In [ ]:
sampler_p = TPESampler(seed = 2021)
study = optuna.create_study(
    study_name = "et_optimizer",
    direction = "minimize",
    sampler = sampler_p,
)
study.optimize(extratree_p, n_trials = 30)

[I 2022-01-22 03:03:48,264] A new study created in memory with name: et_optimizer
[I 2022-01-22 03:03:50,380] Trial 0 finished with value: 32.78235083727571 and parameters: {'n_estimators': 645, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'warm_start': True}. Best is trial 0 with value: 32.78235083727571.
[I 2022-01-22 03:03:51,903] Trial 1 finished with value: 35.91124687181811 and parameters: {'n_estimators': 967, 'max_depth': 13, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'warm_start': True}. Best is trial 0 with value: 32.78235083727571.
[I 2022-01-22 03:03:52,922] Trial 2 finished with value: 35.00136598178736 and parameters: {'n_estimators': 625, 'max_depth': 11, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 5, 'warm_start': False}. Best is trial 0 with value: 32.78235083727571.
[I 2022-01-22 03:03:53,566] Trial 3 finished with value: 34.35443619258099 and parameters: {'n_estimators': 371, 'm

blog data set

b_train_X, b_val_X, b_train_y, b_val_y


In [ ]:
def extratree_b(trial):
  paramset = {
      "n_estimators" : trial.suggest_int("n_estimators", 100, 1000, step = 1),
      "max_depth" : trial.suggest_int("max_depth", 2, 20),
      "min_samples_split":trial.suggest_int("min_samples_split", 2, 4),
      "min_samples_leaf":trial.suggest_int("min_samples_leaf", 1, 3),
      "max_features":trial.suggest_categorical("max_features", ["auto", "sqrt", "log2", 4,5,6]),
      "warm_start":trial.suggest_categorical("warm_start", [True, False]),
      "random_state":2021
  }
  model = ExtraTreesRegressor(**paramset)
  model.fit(b_train_X, b_train_y)
  pred = model.predict(b_val_X)
  rmse = np.sqrt(mean_squared_error(b_val_y, pred))
  return rmse

In [ ]:
sampler_b = TPESampler(seed = 2021)
study_b = optuna.create_study(
    study_name = "et_optimizer",
    direction = "minimize",
    sampler = sampler_b,
)
study_b.optimize(extratree_b, n_trials = 30)

[I 2022-01-22 03:04:14,419] A new study created in memory with name: et_optimizer
[I 2022-01-22 03:04:16,752] Trial 0 finished with value: 36.98123321049414 and parameters: {'n_estimators': 645, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'warm_start': True}. Best is trial 0 with value: 36.98123321049414.
[I 2022-01-22 03:04:18,318] Trial 1 finished with value: 37.55355911702709 and parameters: {'n_estimators': 967, 'max_depth': 13, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'warm_start': True}. Best is trial 0 with value: 36.98123321049414.
[I 2022-01-22 03:04:19,371] Trial 2 finished with value: 37.65208574471828 and parameters: {'n_estimators': 625, 'max_depth': 11, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 5, 'warm_start': False}. Best is trial 0 with value: 36.98123321049414.
[I 2022-01-22 03:04:20,057] Trial 3 finished with value: 37.794316402716724 and parameters: {'n_estimators': 371, '

'n_estimators': 899, 'max_depth': 14, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'warm_start': True

In [ ]:
X = an[an.columns.difference(['id', 'count', 'hour_bef_visibility'])]
colls = X.columns.tolist()
colls

['busy_hour',
 'hour',
 'hour_bef_humidity',
 'hour_bef_ozone',
 'hour_bef_pm10',
 'hour_bef_pm2.5',
 'hour_bef_precipitation',
 'hour_bef_temperature',
 'hour_bef_windspeed']

In [ ]:
#####################
#테스트셋 전처리(결측치, 더미 변수 추가)
if option:
  X = np.column_stack((X['hour']**5, X['hour_bef_temperature']**4, X))
y = an['count']
it_test = test[test.columns.difference(['id', 'hour_bef_visibility'])].copy()
it_test = busyHourGen(it_test, 'hour')
it_test = it_test[colls]
if option:
  X_test = np.column_stack((it_test['hour']**5, it_test['hour_bef_temperature']**4, it_test))
X_test = IterativeImputer(random_state=2021).fit_transform(X_test)

# ExtraTreesRegressor
best_reg = ExtraTreesRegressor(n_estimators=899, max_depth=14, min_samples_split=4, min_samples_leaf=1,max_features='sqrt',warm_start=True, random_state=2021)
best_reg.fit(X, y)
predicts = best_reg.predict(X_test)

In [ ]:
best_reg = ExtraTreesRegressor(n_estimators=899, max_depth=14, min_samples_split=4, min_samples_leaf=1,max_features='sqrt',warm_start=True, random_state=2021)
best_reg.fit(X, y)
predicts = best_reg.predict(X_test)

In [ ]:
X.shape

(1459, 11)

In [ ]:
predicts.shape

(715,)

In [ ]:
submission['count'] = predicts
submission.to_csv('final_et.csv', index=False)

all data set

In [ ]:
f_train_X, f_val_X, f_train_y, f_val_y

(        id  hour  ...  hour_bef_pm2.5  busy_hour
 1346  2028    17  ...            27.0          1
 1273  1928     1  ...             0.0          0
 1382  2073    15  ...            13.0          0
 720   1100     5  ...            18.0          0
 395    601    19  ...            25.0          1
 ...    ...   ...  ...             ...        ...
 621    952    11  ...            42.0          0
 1152  1740    14  ...            30.0          0
 57      91    10  ...            25.0          0
 1365  2052    15  ...            22.0          0
 1140  1721    19  ...            18.0          1
 
 [977 rows x 10 columns],         id  hour  ...  hour_bef_pm2.5  busy_hour
 606    929    19  ...            17.0          1
 616    945     5  ...            24.0          0
 1302  1969     4  ...            39.0          0
 228    359    15  ...            32.0          0
 1412  2118     7  ...            30.0          1
 ...    ...   ...  ...             ...        ...
 857   1301    22  ... 

In [ ]:
def extratree2(trial):
  paramset = {
      "n_estimators" : trial.suggest_int("n_estimators", 100, 1000, step = 1),
      "max_depth" : trial.suggest_int("max_depth", 2, 20),
      "min_samples_split":trial.suggest_int("min_samples_split", 2, 4),
      "min_samples_leaf":trial.suggest_int("min_samples_leaf", 1, 3),
      "max_features":trial.suggest_categorical("max_features", ["auto", "sqrt", "log2", 4,5,6]),
      "warm_start":trial.suggest_categorical("warm_start", [True, False]),
      "random_state":2021
  }
  model = ExtraTreesRegressor(**paramset)
  model.fit(f_train_X, f_train_y)
  pred = model.predict(f_val_X)
  rmse = np.sqrt(mean_squared_error(f_val_y, pred))
  return rmse

In [ ]:
sampler2 = TPESampler(seed = 2021)
study2 = optuna.create_study(
    study_name = "et_optimizer",
    direction = "minimize",
    sampler = sampler2,
)
study2.optimize(extratree2, n_trials = 30)

[I 2022-01-22 03:04:52,071] A new study created in memory with name: et_optimizer
[I 2022-01-22 03:04:53,902] Trial 0 finished with value: 36.62083429045045 and parameters: {'n_estimators': 645, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'warm_start': True}. Best is trial 0 with value: 36.62083429045045.
[I 2022-01-22 03:04:55,338] Trial 1 finished with value: 39.158705800003226 and parameters: {'n_estimators': 967, 'max_depth': 13, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'warm_start': True}. Best is trial 0 with value: 36.62083429045045.
[I 2022-01-22 03:04:56,287] Trial 2 finished with value: 38.360345526078106 and parameters: {'n_estimators': 625, 'max_depth': 11, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 5, 'warm_start': False}. Best is trial 0 with value: 36.62083429045045.
[I 2022-01-22 03:04:56,899] Trial 3 finished with value: 37.95162544283364 and parameters: {'n_estimators': 371, 

best 
::
'n_estimators': 645, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'warm_start': True

In [ ]:
bestmodel = ExtraTreesRegressor(n_estimators=645, max_depth=15, min_samples_split=2, min_samples_leaf=1, max_features='auto', warm_start=True)
bestmodel.fit(final_x, final_y)
pred = bestmodel.predict(test_pro)

In [ ]:
submission = pd.read_csv('/content/data/submission.csv')
submission['count'] = pred
submission.to_csv('베이스라인.csv',index = False)
submission

,id,count
0,0,103.128839
1,1,260.155182
2,2,44.247244
3,4,38.695002
4,5,49.209302
...,...,...
710,2148,75.303204
711,2149,67.124512
712,2165,120.869492
713,2166,140.554346


##GradientBoosting

In [ ]:
#f_train_X, f_val_X, f_train_y, f_val_y

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

GBR 모델

In [ ]:
reg = GradientBoostingRegressor(random_state=0)
reg.fit(f_train_X, f_train_y)
pred = reg.predict(f_val_X)
rmse = np.sqrt(mean_squared_error(f_val_y, pred))
print(rmse)

38.16010514484238


In [ ]:
def GB(trial):
  paramset = {
      "loss" : "squared_error",
      "n_estimators" : trial.suggest_int("n_estimators", 100, 1000, step = 1),
      "max_depth" : trial.suggest_int("max_depth", 2, 20),
      "min_samples_split":trial.suggest_int("min_samples_split", 2, 4),
      "min_samples_leaf":trial.suggest_int("min_samples_leaf", 1, 3),
      "max_features":trial.suggest_categorical("max_features", ["auto", "sqrt", "log2", 4,5,6]),
      "warm_start":trial.suggest_categorical("warm_start", [True, False]),
  }
  model = GradientBoostingRegressor(**paramset)
  model.fit(f_train_X, f_train_y)
  pred = model.predict(f_val_X)
  rmse = np.sqrt(mean_squared_error(f_val_y, pred))
  return rmse

In [ ]:
sampler3 = TPESampler(seed = 2021)
study3 = optuna.create_study(
    study_name = "et_optimizer",
    direction = "minimize",
    sampler = sampler3,
)
study3.optimize(GB, n_trials = 30)

[I 2022-01-22 03:05:20,845] A new study created in memory with name: et_optimizer
[I 2022-01-22 03:05:22,207] Trial 0 finished with value: 47.41058668372477 and parameters: {'n_estimators': 645, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'warm_start': True}. Best is trial 0 with value: 47.41058668372477.
[I 2022-01-22 03:05:23,932] Trial 1 finished with value: 39.834355546089036 and parameters: {'n_estimators': 967, 'max_depth': 13, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'warm_start': True}. Best is trial 1 with value: 39.834355546089036.
[I 2022-01-22 03:05:25,596] Trial 2 finished with value: 39.441339065780355 and parameters: {'n_estimators': 625, 'max_depth': 11, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 5, 'warm_start': False}. Best is trial 2 with value: 39.441339065780355.
[I 2022-01-22 03:05:27,019] Trial 3 finished with value: 38.538617498322196 and parameters: {'n_estimators': 37

blog data

b_train_X, b_val_X, b_train_y, b_val_y


best_reg.fit(X, y)
predicts = best_reg.predict(X_test)

In [ ]:
def B_GB(trial):
  paramset = {
      "loss" : "squared_error",
      "n_estimators" : trial.suggest_int("n_estimators", 100, 1000, step = 1),
      "max_depth" : trial.suggest_int("max_depth", 2, 20),
      "min_samples_split":trial.suggest_int("min_samples_split", 2, 4),
      "min_samples_leaf":trial.suggest_int("min_samples_leaf", 1, 3),
      "max_features":trial.suggest_categorical("max_features", ["auto", "sqrt", "log2", 4,5,6]),
      "warm_start":trial.suggest_categorical("warm_start", [True, False]),
  }
  model = GradientBoostingRegressor(**paramset)
  model.fit(b_train_X, b_train_y)
  pred = model.predict(b_val_X)
  rmse = np.sqrt(mean_squared_error(b_val_y, pred))
  return rmse

In [ ]:
sampler__b = TPESampler(seed = 2021)
study__b = optuna.create_study(
    study_name = "et_optimizer",
    direction = "minimize",
    sampler = sampler__b,
)
study__b.optimize(B_GB, n_trials = 30)

[I 2022-01-22 03:05:57,060] A new study created in memory with name: et_optimizer
[I 2022-01-22 03:05:58,996] Trial 0 finished with value: 49.445244339813286 and parameters: {'n_estimators': 645, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'warm_start': True}. Best is trial 0 with value: 49.445244339813286.
[I 2022-01-22 03:06:00,472] Trial 1 finished with value: 38.2651088387738 and parameters: {'n_estimators': 967, 'max_depth': 13, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'warm_start': True}. Best is trial 1 with value: 38.2651088387738.
[I 2022-01-22 03:06:02,589] Trial 2 finished with value: 36.47098962061164 and parameters: {'n_estimators': 625, 'max_depth': 11, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 5, 'warm_start': False}. Best is trial 2 with value: 36.47098962061164.
[I 2022-01-22 03:06:04,297] Trial 3 finished with value: 38.33377264080138 and parameters: {'n_estimators': 371, 'm

n_estimators': 281, 'max_depth': 9, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 'log2', 'warm_start': True`

In [ ]:
best_reg = GradientBoostingRegressor(n_estimators=281, max_depth=9, min_samples_split=2, min_samples_leaf=3,max_features='log2',warm_start=True, random_state=2021)
best_reg.fit(X, y)
predicts = best_reg.predict(X_test)

In [ ]:
submission['count'] = predicts
submission.to_csv('final_gbr.csv', index=False)

n_estimators': 559, 'max_depth': 3, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 6, 'warm_start': True}

In [ ]:
bestmodel = GradientBoostingRegressor(n_estimators=559, max_depth=3, min_samples_split=3, min_samples_leaf=3, max_features=6, warm_start=True)
bestmodel.fit(final_x, final_y)
pred = bestmodel.predict(test_pro)

In [ ]:
submission = pd.read_csv('/content/data/submission.csv')
submission['count'] = pred
submission.to_csv('baseset_GBR.csv',index = False)
submission

,id,count
0,0,133.495064
1,1,235.390618
2,2,45.604950
3,4,63.640580
4,5,81.195435
...,...,...
710,2148,82.145006
711,2149,46.707589
712,2165,111.522325
713,2166,158.549236


##RandomForestRegressor

In [ ]:

def randomforest(trial):
  # Create the random grid
  params_rf = {
      'n_estimators': trial.suggest_int('n_estimaters', 100,1000, step=1),
      'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt','log2',4,5,6]),
      'max_depth': trial.suggest_int('max_depth', 2,20),
      'min_samples_split': trial.suggest_int('min_samples_split', 2,4),
      'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1,4),
      'bootstrap': trial.suggest_categorical('bootstrap',[True, False]),
}
  model = RandomForestRegressor(**params_rf)
  model.fit(train_pro_x, train_pro_y)

  pred = model.predict(test_pro_x)
  rmse = np.sqrt(mean_squared_error(test_pro_y,pred))
  return rmse


b_train_X, b_val_X, b_train_y, b_val_y

In [ ]:

def randomforest_2(trial):
  # Create the random grid
  params_rf = {
      'n_estimators': trial.suggest_int('n_estimaters', 100,1000, step=1),
      'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt','log2',4,5,6]),
      'max_depth': trial.suggest_int('max_depth', 2,20),
      'min_samples_split': trial.suggest_int('min_samples_split', 2,4),
      'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1,4),
      'bootstrap': trial.suggest_categorical('bootstrap',[True, False]),
}
  model = RandomForestRegressor(**params_rf)
  model.fit(train_x, train_y)

  pred = model.predict(test_x)
  rmse = np.sqrt(mean_squared_error(test_y,pred))
  return rmse


In [ ]:
sampler_rt = TPESampler(seed=2022)
study_rt = optuna.create_study(
    study_name = 'rf',
    direction='minimize',
    sampler = sampler_rt
)
study_rt.optimize(randomforest, n_trials=30)

[I 2022-01-22 03:06:50,939] A new study created in memory with name: rf
[I 2022-01-22 03:06:51,352] Trial 0 finished with value: 34.10942086594019 and parameters: {'n_estimaters': 108, 'max_features': 6, 'max_depth': 14, 'min_samples_split': 4, 'min_samples_leaf': 3, 'bootstrap': True}. Best is trial 0 with value: 34.10942086594019.
[I 2022-01-22 03:06:53,077] Trial 1 finished with value: 41.88433280448068 and parameters: {'n_estimaters': 851, 'max_features': 6, 'max_depth': 3, 'min_samples_split': 4, 'min_samples_leaf': 2, 'bootstrap': False}. Best is trial 0 with value: 34.10942086594019.
[I 2022-01-22 03:06:53,365] Trial 2 finished with value: 36.46926038814867 and parameters: {'n_estimaters': 116, 'max_features': 'sqrt', 'max_depth': 8, 'min_samples_split': 4, 'min_samples_leaf': 1, 'bootstrap': True}. Best is trial 0 with value: 34.10942086594019.
[I 2022-01-22 03:06:55,208] Trial 3 finished with value: 36.48589704658527 and parameters: {'n_estimaters': 773, 'max_features': 'log2'